# your prompt:
As a business analyst, help me write a SQL query to identify active users who have logged in the last 30 days, filter for high value customers who 
are in the top 20% of total amount spent over the past year,
and for those top 20% of spenders identify trends in their spending preferences, looking at when they bought things and on which platform

Database Schema:

user_activity
    user_id
    last_login_date
    feature_usage_count
    account_type
transactions
    transaction_id
    user_id
    transaction_date
    amount
    platform
user_preferences
    user_id
    communication_preference
    interface_theme
    notification_settings

In [ ]:
asked the AI to identify users who logged in the past 30 days, who are in the top20% of spenders over the past year, asked to look
at their spending preferences, when they bought things and on which platform

In [2]:
# generated code:

WITH active_users AS (
    SELECT user_id
    FROM user_activity
    WHERE last_login_date >= CURRENT_DATE - INTERVAL '30 days'
),

yearly_spending AS (
    SELECT 
        user_id,
        SUM(amount) AS total_spent
    FROM transactions
    WHERE transaction_date >= CURRENT_DATE - INTERVAL '1 year'
    GROUP BY user_id
),

top_20_percent_spenders AS (
    SELECT user_id
    FROM (
        SELECT 
            user_id,
            total_spent,
            NTILE(5) OVER (ORDER BY total_spent DESC) AS spending_bucket
        FROM yearly_spending
    ) ranked
    WHERE spending_bucket = 1
),

high_value_active_users AS (
    SELECT au.user_id
    FROM active_users au
    INNER JOIN top_20_percent_spenders tp ON au.user_id = tp.user_id
)

SELECT 
    t.user_id,
    DATE_TRUNC('month', t.transaction_date) AS month,
    t.platform,
    COUNT(t.transaction_id) AS num_transactions,
    SUM(t.amount) AS total_spent
FROM transactions t
JOIN high_value_active_users hv ON t.user_id = hv.user_id
WHERE t.transaction_date >= CURRENT_DATE - INTERVAL '1 year'
GROUP BY t.user_id, month, t.platform
ORDER BY t.user_id, month;


# follow up prompts:
how do those customers prefer to be communicated with, what kind of notification settings do they have on,
is there a preferred interface thats common, is there a preferred platform they use

asked about their preferred communication methods, when they want to be notified, and is there preferred platform and interface

In [4]:
# final solution:
WITH active_users AS (
    SELECT user_id
    FROM user_activity
    WHERE last_login_date >= CURRENT_DATE - INTERVAL '30 days'
),

yearly_spending AS (
    SELECT 
        user_id,
        SUM(amount) AS total_spent
    FROM transactions
    WHERE transaction_date >= CURRENT_DATE - INTERVAL '1 year'
    GROUP BY user_id
),

top_20_percent_spenders AS (
    SELECT user_id
    FROM (
        SELECT 
            user_id,
            total_spent,
            NTILE(5) OVER (ORDER BY total_spent DESC) AS spending_bucket
        FROM yearly_spending
    ) ranked
    WHERE spending_bucket = 1
),

high_value_active_users AS (
    SELECT au.user_id
    FROM active_users au
    INNER JOIN top_20_percent_spenders tp ON au.user_id = tp.user_id
),

-- Platform usage ranking for each user
platform_usage AS (
    SELECT
        user_id,
        platform,
        COUNT(*) AS platform_count,
        RANK() OVER (PARTITION BY user_id ORDER BY COUNT(*) DESC) AS platform_rank
    FROM transactions
    WHERE transaction_date >= CURRENT_DATE - INTERVAL '1 year'
    GROUP BY user_id, platform
),

-- Preferences + platform of high-value active users
user_details AS (
    SELECT 
        h.user_id,
        up.communication_preference,
        up.notification_settings,
        up.interface_theme,
        pu.platform AS preferred_platform
    FROM high_value_active_users h
    LEFT JOIN user_preferences up ON h.user_id = up.user_id
    LEFT JOIN platform_usage pu 
        ON h.user_id = pu.user_id AND pu.platform_rank = 1
)

-- Final spending trend + enriched user data
SELECT 
    t.user_id,
    ud.communication_preference,
    ud.notification_settings,
    ud.interface_theme,
    ud.preferred_platform,
    DATE_TRUNC('month', t.transaction_date) AS month,
    t.platform,
    COUNT(t.transaction_id) AS num_transactions,
    SUM(t.amount) AS total_spent
FROM transactions t
JOIN high_value_active_users hv ON t.user_id = hv.user_id
LEFT JOIN user_details ud ON t.user_id = ud.user_id
WHERE t.transaction_date >= CURRENT_DATE - INTERVAL '1 year'
GROUP BY 
    t.user_id,
    ud.communication_preference,
    ud.notification_settings,
    ud.interface_theme,
    ud.preferred_platform,
    month,
    t.platform
ORDER BY t.user_id, month;

Discussion Points:

How were percentile calculations handled?
What approaches to date filtering were used?
How was the query optimized, CTE, subquery etc…?